In [1]:
import pandas as pd 
import sqlite3

In [2]:
# table_dict = {1 : 'aisles', 2: 'departments', 3: 'order_details', 4: 'orders', 6: 'products'}

 
# for table in table_dict:
#     table_name = table
#     statement = f'select * from {table_dict[table]}'
#     conn = sqlite3.Connection('instacart.db')
#     table_dict[table] = pd.read_sql(statement, conn)
#     print(statement)

# aisles_df = table_dict[1]
# departments_df = table_dict[2]
# order_details_df = table_dict[3]
# orders_df = table_dict[4]
# products_df = table_dict[6]


# aisles_df.to_csv('..\\capstone_project\\aisles.csv', index = False)
# departments_df.to_csv('..\\capstone_project\\departments.csv', index = False)

# order_details_df.to_csv('..\\capstone_project\\order_details.csv', index = False)

# orders_df.to_csv('..\\capstone_project\\orders.csv', index = False)

# products_df.to_csv('..\\capstone_project\\products.csv', index = False)

## Create data structure for gensim

In [3]:
order_products = pd.read_csv('order_details.csv')
orders_df = pd.read_csv('orders.csv')
products_df = pd.read_csv('products.csv')

In [4]:
order_product_name = pd.merge(order_products, products_df, how = 'left', on ='product_id')
user_order_product = pd.merge(order_product_name, orders_df, how = 'left', on = 'order_id')
user_order_product['product_name'][0]

'Organic Egg Whites'

Replace ' ' with underscores (_)

In [5]:
products = user_order_product['product_name']
product_no_space = []
for product in products:
    product = product.replace(' ', '_')
    product_no_space.append(product)
    
#Drop original column, and replace it with one with no space 
user_order_product.drop(['product_name'], axis = 1)
user_order_product['product_name'] = product_no_space

In [6]:
user_order_product.shape

(32434489, 13)

Now we want to add cluster data to user_order_product so we can create bigrams for each cluster

In [7]:
clustered_users = pd.read_csv('clustered_users.csv')

In [8]:
# drop unnecessary columns 
clustered_users.columns
clustered_users = clustered_users.drop(columns = ['Unnamed: 0',  'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'num_orders', 'num_products',
       'avg_num_products_per_order', 'X_coor', 'Y_coor'])
clustered_users.head()

,user_id,Clusters
0,1,6
1,2,10
2,3,1
3,4,6
4,5,1


In [9]:
user_product_order = pd.merge(user_order_product, clustered_users, how = 'inner', on = 'user_id')
user_product_order.head()

,id,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,Clusters
0,14111,1483,32818,1,1,Organic_Pink_Lemonade_Bunny_Fruit_Snacks,50,19,90,31,3,13,5.0,11
1,14112,1483,27582,2,1,Dark_Chocolate_Minis,45,19,90,31,3,13,5.0,11
2,14113,1483,12302,3,1,"Sparkling_Water,_Natural_Mango_Essenced",115,7,90,31,3,13,5.0,11
3,14114,1483,1831,4,1,Peach-Pear_Sparkling_Water,115,7,90,31,3,13,5.0,11
4,14115,1483,19204,5,1,Organic_Heritage_Flakes_Cereal,121,14,90,31,3,13,5.0,11


Now we want to combine all user products into a single row

In [10]:
user_product_dict_int_keys = {}
name_list = []

# create user:products dictionary
for p_name in user_product_order.groupby('user_id')['product_name']:
    name_list.append(p_name[1])
for user in range(len(name_list)):
    user_product_dict_int_keys[user] = list(name_list[user])

# create cluster:products dictionary
cluster_dict_int_keys = {}
cluster_list = []

for p_name in user_product_order.groupby('Clusters')['product_name']:
    cluster_list.append(p_name[1])

for cluster in range(len(cluster_list)):
    cluster_dict_int_keys[cluster] = list(cluster_list[cluster])


### Generate Bigrams - all user data, cluster data

In [11]:
import gensim

In [36]:
# all user bigrams
bigram_phrases = gensim.models.Phrases(user_product_dict_int_keys.values(), min_count =2, threshold = 5, delimiter = ' ')
bigram = gensim.models.phrases.Phraser(bigram_phrases)
user_bigram = bigram.phrasegrams

# cluster bigrams
cluster_dict = {}
temporary_cluster_dict = {}

for key, value in cluster_dict_int_keys.items():
    temp_key = 'Cluster_' + str(key)
    key1 = 0
    temporary_cluster_dict[key1] = cluster_dict_int_keys[key]
    if key == 9:
        cluster_phrases = gensim.models.Phrases(temporary_cluster_dict.values(), min_count = 2, threshold = 5, delimiter = ' ')
        cluster_bigram = gensim.models.phrases.Phraser(cluster_phrases)
        cluster_dict[temp_key] = cluster_bigram.phrasegrams
    else:
        cluster_phrases = gensim.models.Phrases(temporary_cluster_dict.values(), min_count = 3, threshold = 15, delimiter = ' ')
        cluster_bigram = gensim.models.phrases.Phraser(cluster_phrases)
        cluster_dict[temp_key] = cluster_bigram.phrasegrams
    

### Now organize the bigrams by the first word 

We want a dictionary that has all user bigrams where the first word in the bigram is the key and the value is all of the other products that follow that word as well as their bigram score <br>
ex: `{'soda':{water: 10}, {milk: 8}}`

We also want a dictionary seperated by clusters and their bigrams <br>
ex: `{cluster_1: {'soda':{water: 10}, {milk: 8}}}` <br>
`{cluster_2: {'cereal':{milk: 10}, {bananas: 8}}}`

This is hard because looping through the dictionary without additional logic overwrites key : value pairs so we are stuck with only one key value pair for each key, when we should have a few products in each cluster with multiple values. *see soda with multiple values (water & milk)

### Every user order bigram combined into one dictionary 

In [37]:
# Creates User Bigram Dictionary (Organized by first word)

user_bigram_dict = {}
items = []

for key in user_bigram:
    items.append(key.split())

bigram_vals = list(user_bigram.values())

count = 0
for item in items:
    if item[0] in user_bigram_dict.keys():
        user_bigram_dict[item[0]][item[1]] = int(bigram_vals[count])
    else:
        user_bigram_dict[item[0]] = {}
        user_bigram_dict[item[0]][item[1]] = int(bigram_vals[count])
    count+=1

### All user order bigrams organized by cluster 

In [54]:
final_cluster = {}

temp_cluster = {}
for cluster, value in cluster_dict.items():
    temp_cluster = value
    intermediate_cluster = {'': []}
    for key, value1 in temp_cluster.items():
        key1, key2 = key.split()
        pairing = {key2: value1}
        if key1 in intermediate_cluster.keys():
            key1_list = list(intermediate_cluster[key1].items())
            key1_list.append(pairing)
            intermediate_cluster[key1] = key1_list
        else:  
            intermediate_cluster[key1] = pairing
    intermediate_cluster.pop('')
    final_cluster[cluster] = intermediate_cluster
   

In [55]:
final_cluster['Cluster_1']['Soda']

[('Sparkling_Mineral_Water', 28.520833333333332),
 {'Smartwater': 22.816666666666666}]

### Time to sort the user bigrams by frequency score

In [56]:
from operator import itemgetter
# create temp dict, to not mess up user_bigram_dict
user_temp_dict = user_bigram_dict

# this will be the new dictionary of bigrams, the exact same but sorted by frequency score
user_bigrams_sorted = {}

for key in user_temp_dict:
#     print(key)
    user_bigrams_sorted[key] = dict(sorted(user_temp_dict[key].items(), key = itemgetter(1), reverse = True)) # <- returns a list so had to parse into a dictionary 


### Export Cluster dictionaries 

In [59]:
import json
with open('all_bigrams.json', 'w') as file:
    json.dump(user_bigrams_sorted, file)

In [60]:
with open('all_cluster_bigrams.json', 'w') as file:
    json.dump(final_cluster, file)